# Model parameter estimation for an irregularly-sampled, continuous-discrete (non-linear) dynamical system

We show how to use cd-dynamax to estimate the parameters of a continuous-discrete (non-linear) dynamical system



## Preliminaries

### Code Setup

In [ ]:
# Main imports
import sys
from itertools import count

# Import jax and utils
from jax import numpy as jnp
from jax import vmap
import jax.random as jr
from jax import jit, vmap

# Additional, custom codebase
sys.path.append("../..")
sys.path.append("../../..")

# Our own custom src codebase
# continuous-discrete nonlinear Gaussian SSM codebase
from utils.plotting_utils import *
# CD-Nonlinear Gaussian models
from continuous_discrete_nonlinear_gaussian_ssm import ContDiscreteNonlinearGaussianSSM
from continuous_discrete_nonlinear_gaussian_ssm.models import *

# Useful utility functions
from simulation_utils import *

# Plotting
import matplotlib
%matplotlib inline
# Our own custom plotting codebase
from utils.plotting_utils import *
from lorenz_plotting import *
from parameter_learning_plotting import *
# Feel free to change the default figure size
#matplotlib.rcParams['figure.figsize'] = [16, 9]


# Generate synthetic data

We generate data from a Lorenz 63 system, from dynamics with the following stochastic differential equations:
\begin{align*}
\frac{d x}{d t} &= a(y-x) + \sigma w_x(t) \\
\frac{d y}{d t} &= x(b-z) - y + \sigma w_y(t) \\
\frac{d z}{d t} &= xy - cz + \sigma w_z(t),
\end{align*}

With parameters $a=10, b=28, c=8/3$, the system gives rise to chaotic behavior, and we choose $\sigma=0.1$ for light diffusion.

To generate data, we numerically approximate random path solutions to this SDE using Heun's method (i.e. improved Euler), as implemented in [Diffrax](https://docs.kidger.site/diffrax/api/solvers/sde_solvers/).


We assume the observation model is
\begin{align*}
y(t) &= H x(t) + r(t) \\
r(t) &\sim N(0,R),
\end{align*}
where we choose $R=I$. 

Namely, with H=I, we have full observability, with noisy observations, sampled at irregular time intervals.

### True, data-generating model definition

In [2]:
## Main settings
state_dim = 3
emission_dim = 3

# Define a custom drift model, inherited from LearnableFunction
class lorenz63_drift(LearnableFunction):
    params: Union[Float[Array, "state_dim"], ParameterProperties]

    def f(self, x, u=None, t=None):
        foo = jnp.array(
            [
                self.params[0] * (x[1] - x[0]),
                self.params[1] * x[0] - x[1] - x[0] * x[2],
                -self.params[2] * x[2] + x[0] * x[1],
            ]
        )
        return foo

# Define the true parameters of the drift function
true_l63_drift_params = jnp.array([10.0, 28.0, 8 / 3])
# And the corresponding Lorenz 63 system
true_drift = {
    "params": lorenz63_drift(
        params=true_l63_drift_params
    ),
    "props": lorenz63_drift(
        params=ParameterProperties()
    ),
}

# Define the true parameters of the diffusion function
true_diffusion_cov = {
    "params": LearnableMatrix(
        params=jnp.eye(state_dim)
    ),
    "props": LearnableMatrix(
        params=ParameterProperties(
            constrainer=RealToPSDBijector()
        )
    ),
}

# Define the true parameters of the diffusion function
true_diffusion_coefficient_param = 0.1
true_diffusion_coefficient = {
    "params": LearnableMatrix(
        params=true_diffusion_coefficient_param * jnp.eye(state_dim)
    ),
    "props": LearnableMatrix(
        params=ParameterProperties()
    ),
}

# Define the true parameters of the emission function
# Full observability
H=jnp.eye(emission_dim,state_dim)
true_emission = {
    "params": LearnableLinear(
        weights=H,
        bias=jnp.zeros(emission_dim)
    ),
    "props": LearnableLinear(
        weights=ParameterProperties(),
        bias=ParameterProperties()
    ),
}

# Define the true parameters of the emission covariance
R=jnp.eye(emission_dim)
true_emission_cov = {
    "params": LearnableMatrix(
        params=R
    ),
    "props": LearnableMatrix(
        params=ParameterProperties(
            constrainer=RealToPSDBijector()
        )
    ),
}

# Define the true initial mean and covariance
true_initial_mean = {
    "params": LearnableVector(
        params=jnp.zeros(state_dim)
    ),
    "props": LearnableVector(
        params=ParameterProperties()
    ),
}

true_initial_cov_param = 10.0
true_initial_cov = {
    "params": LearnableMatrix(
        params=true_initial_cov_param*jnp.eye(state_dim)
    ),
    "props": LearnableMatrix(
        params=ParameterProperties(
            constrainer=RealToPSDBijector()
        )
    ),
}

# Concatenate all parameters in dictionary, for later easy use
all_true_params = {
    'initial_mean': true_initial_mean,
    'initial_cov': true_initial_cov,
    'dynamics_drift': true_drift,
    'dynamics_diffusion_coefficient': true_diffusion_coefficient,
    'dynamics_diffusion_cov': true_diffusion_cov,
    'dynamics_approx_order': 2., # Check on this later
    'emission_function': true_emission,
    'emission_cov': true_emission_cov,
}

### Simulation set-up

In [3]:
# Set up seed for simulation
keys = map(jr.PRNGKey, count())

# Simulation parameters
T_filter = 40
T_total = 50
num_timesteps_total = int(T_total*100)

# Generate time points for measurements, filtering and forecasting
# We will generate different train and test sets

## Train set
t_emissions_train, t_filter_train, t_forecast_train, \
    num_timesteps_train, num_timesteps_filter_train, num_timesteps_forecast_train= \
    generate_irregular_t_emissions(
        T_total=T_total,
        num_timesteps=num_timesteps_total,
        T_filter=T_filter,
        key=next(keys)
    )

## Test set
t_emissions_test, t_filter_test, t_forecast_test, \
    num_timesteps_test, num_timesteps_filter_test, num_timesteps_forecast_test= \
    generate_irregular_t_emissions(
        T_total=T_total,
        num_timesteps=num_timesteps_total,
        T_filter=T_filter,
        key=next(keys)
    )

### Model creation: object instantiation

In [4]:
# Create CD-NLGSSM model
true_model = ContDiscreteNonlinearGaussianSSM(state_dim, emission_dim)
true_params, _ = true_model.initialize(
    next(keys),
    **all_true_params
)

### Simulate data: sample from model

In [ ]:
# sample true states and emissions from defined true model.
# Using transition_type="path" to solve the dynamics SDE.

## Train set
true_states_train, true_emissions_train = true_model.sample(
    true_params,
    next(keys),
    num_timesteps_train,
    t_emissions_train,
    transition_type="path" # uses the Euler-Maruyama method
)

## Test set
true_states_test, true_emissions_test = true_model.sample(
    true_params,
    next(keys),
    num_timesteps_test,
    t_emissions_test,
    transition_type="path" # uses the Euler-Maruyama method
)

### Plot generated data

In [ ]:
# Plot all the training data
plot_advanced2(
    time_grid_all=t_emissions_train,
    true_states=true_states_train,
    true_emissions_noisy=true_emissions_train,
    emission_function=true_params.emissions.emission_function
)

# Plot the filtering training data
plot_advanced2(
    time_grid_all=t_emissions_train,
    true_states=true_states_train,
    true_emissions_noisy=true_emissions_train,
    emission_function=true_params.emissions.emission_function,
    t_start=0,
    t_end=T_filter
)

# Plot the forecasting training data
plot_advanced2(
    time_grid_all=t_emissions_train,
    true_states=true_states_train,
    true_emissions_noisy=true_emissions_train,
    emission_function=true_params.emissions.emission_function,
    t_start=T_filter,
    t_end=T_total
)

# Parameter learning

### We start by defining a new model that is learnable

- We want to learn a model that might or might not be within the same function class of the data-generating mechanism

- The learnable model will be a ContDiscreteNonlinearGaussianSSM

- Here, we define the same model family as the data-generating model
    - i.e., the model family class is the same

- However:
    - the dynamics drift function will not-initialized to true values, and will be learnable
    - The rest of the model components will NOT be learnable

### CODE hint

- By default, all ParamProperties are set to trainable=True

- So, make sure you EXPLICITLY set trainable=False for all parameters you DO NOT want to learn/infer

In [7]:
## Main settings
state_dim = 3
emission_dim = 3

# The only learnable parameters are those in the learnable_drift
# An initial guess that is close to the true parameters
perturbation = jr.normal(next(keys), (len(true_l63_drift_params),))
drift_parameter_guess = true_l63_drift_params * (1.0 + 0.5 * perturbation)

# Define the learnable drift model, with initial guess
learnable_drift = {
    "params": lorenz63_drift(params=drift_parameter_guess),
    "props": lorenz63_drift(params=ParameterProperties()),
}

## All parameters below are NOT learnable
learnable_diffusion_cov = {
    "params": LearnableMatrix(
        params=jnp.eye(state_dim)
    ),
    "props": LearnableMatrix(
        params=ParameterProperties(
            trainable=False,
            constrainer=RealToPSDBijector()
        )
    ),
}

# The diffusion coefficient is fixed to non-learnable identity
learnable_diffusion_coefficient = {
    "params": LearnableMatrix(
        params=true_diffusion_coefficient_param * jnp.eye(state_dim)
    ),
    "props": LearnableMatrix(
        params=ParameterProperties(trainable=False)
    ),
}

# The emission function is fixed to non-learnable identity
learnable_emission = {
    "params": LearnableLinear(
        weights=H,
        bias=jnp.zeros(emission_dim)
    ),
    "props": LearnableLinear(
        weights=ParameterProperties(trainable=False),
        bias=ParameterProperties(trainable=False)
    ),
}

# The emission covariance is fixed to non-learnable identity
learnable_emission_cov = {
    "params": LearnableMatrix(
        params=R
    ),
    "props": LearnableMatrix(
        params=ParameterProperties(
            trainable=False,
            constrainer=RealToPSDBijector()
        )
    ),
}

# Define the initial mean and covariance
learnable_initial_mean = {
    "params": LearnableVector(
        params=jnp.zeros(state_dim)
    ),
    "props": LearnableVector(
        params=ParameterProperties(trainable=False)
    ),
}

learnable_initial_cov = {
    "params": LearnableMatrix(
        params=10.0 * jnp.eye(state_dim)
    ),
    "props": LearnableMatrix(params=ParameterProperties(trainable=False, constrainer=RealToPSDBijector())),
}

# Concatenate all parameters in dictionary, for later easy use
all_learnable_params = {
    "initial_mean": learnable_initial_mean,
    "initial_cov": learnable_initial_cov,
    "dynamics_drift": learnable_drift,
    "dynamics_diffusion_coefficient": learnable_diffusion_coefficient,
    "dynamics_diffusion_cov": learnable_diffusion_cov,
    "dynamics_approx_order": 2.,  # Check on this later
    "emission_function": learnable_emission,
    "emission_cov": learnable_emission_cov,
}

### Define, instantiate and initialize the learnable model

In [8]:
# Create CD-NLGSSM model
learnable_model = ContDiscreteNonlinearGaussianSSM(state_dim, emission_dim)
initial_learnable_params, learnable_props = learnable_model.initialize(
    next(keys),
    **all_learnable_params
)

# Bayesian parameter estimation: Maximum A-Posteriori (MAP) estimates

## Bayesian parameter estimation for an irregularly-sampled, continuous-discrete (non-linear) Gaussian dynamical system

### HMC w/ autodifferentiable filtering

We show how to use the [blackjax](https://github.com/blackjax-devs/blackjax) libray to compute the HMC-based parameter posterior $p(\theta|y(1:T))$ for a nonlinear dynamical model.

    - TODO: Showcase how to use NUTS

We use the Extended Kalman filter to compute the marginal likelihood, $p(y(1:T) | \theta) = \int_{z(1:T)} p(z(1:T), y(1:T)|\theta)$.


### Model MAP learning, based on HMC
- We will use Extended Kalman Filtering to compute the marginal log probability, for SGD to optimize

In [9]:
# Filter: EKF
hmc_filter_hyperparams = EKFHyperParams()

### HMC hyperparameters and initialization

In [10]:
# HMC hyperparameters (100/100/15 runs in ~15minutes. For faster run, reduce these numbers)
hmc_N_samples = 100
hmc_warmup = 100
hmc_integration_steps=15

In [ ]:
# HMC is initialized at random parameters
# Run
hmc_param_samples, hmc_marginal_log_prob = learnable_model.fit_hmc(
    initial_learnable_params,
    learnable_props,
    true_emissions_train,
    t_emissions=t_emissions_train,
    filter_hyperparams=hmc_filter_hyperparams,
    num_samples=hmc_N_samples,
    warmup_steps=hmc_warmup,
    num_integration_steps=hmc_integration_steps,
)

### Visualize learning: marginal log-likelihood evolution over MCMC iterations

In [ ]:
plot_mll_learning_curve(
    true_model,
    true_params,
    true_emissions_train,
    t_emissions_train,
    hmc_marginal_log_prob,
)

In [13]:
# Define filter and forecast time points
T0 = 5 # Ignore initial transient
T_filter_end = 25 # End of filtering
T_forecast_end = 45 # End of forecasting

In [14]:
# vmap over filter and forecast function
(hmc_filtered_train, hmc_forecasted_train, \
    hmc_start_idx_filter, hmc_stop_idx_filter, \
    hmc_start_idx_forecast, hmc_stop_idx_forecast
    ) = vmap(
        filter_and_forecast,
        in_axes=(0, None, None, None, None, None, None) # only over param_samples
    )(
        hmc_param_samples,
        hmc_filter_hyperparams,
        t_emissions_train,
        true_emissions_train,
        T0,
        T_filter_end,
        T_forecast_end
    )


In [15]:
# vmap over cdnlssm emissions function, to get the filtered emissions
# based on the vmapped params, filtered state means and filtered state covariances
(hmc_filtered_train_emissions_mean, hmc_filtered_train_emissions_covariance) = vmap(
        cdnlgssm_emissions,
        in_axes=(0, None, 0, 0, None, None, None) # vmap over axis 0 of params, state_means and state_covs
    )(
        hmc_param_samples,
        t_emissions_train[hmc_start_idx_filter[0]:hmc_stop_idx_filter[0]],
        hmc_filtered_train.filtered_means,
        hmc_filtered_train.filtered_covariances,
        None,
        EKFHyperParams(),
        next(keys),
    )

In [16]:
# vmap over cdnlssm emissions function, to get the forecasted emissions
# based on the vmapped params, forecasted state means and state covariances
(hmc_forecasted_train_emissions_mean, hmc_forecasted_train_emissions_covariance) = vmap(
        cdnlgssm_emissions,
        in_axes=(0, None, 0, 0, None, None, None) # vmap over axis 0 of params, state_means and state_covs
    )(
        hmc_param_samples,
        t_emissions_train[hmc_start_idx_forecast[0]:hmc_stop_idx_forecast[0]],
        hmc_forecasted_train.forecasted_state_means,
        hmc_forecasted_train.forecasted_state_covariances,
        None,
        EKFHyperParams(),
        next(keys),
    )

In [17]:
start_idx_filter_hmc = hmc_start_idx_filter[0]
stop_idx_filter_hmc = hmc_stop_idx_filter[0]
start_idx_forecast_hmc = hmc_start_idx_forecast[0]
stop_idx_forecast_hmc = hmc_stop_idx_forecast[0]

# Auxiliary function to split parameter samples into a list
def split_params(params, index):
    """Extracts the specific index from each parameter in the structure."""
    return jax.tree_map(lambda x: x[index], params)

# Split the emission functions into a list
emission_function_list = [split_params(hmc_param_samples.emissions.emission_function, i) for i in range(hmc_N_samples)]
# now extract the .f function from each of the emission functions
emission_function_f_list = [x.f for x in emission_function_list]

### Plot performance of filtering and forecasting on training data

In [ ]:
# Plot the true states and emissions, and the EKF estimates
for t_start, t_end in [
        (None, None), # Plot all
        (T0, T_forecast_end), # Plot from T0 until T_forecast_end, i.e., filtering and forecasting
        (T0, T_filter_end), # Plot from T0 until T_forecast_end, i.e., filtering
        (T_filter_end, T_forecast_end) # Plot from T_filter_end until T_forecast_end, i.e., forecasting
    ]:
    plot_advanced2(
        time_grid_all=t_emissions_train,
        true_states=true_states_train,
        true_emissions_noisy=true_emissions_train,
        emission_function=emission_function_f_list,
        time_grid_filter=t_emissions_train[start_idx_filter_hmc:stop_idx_filter_hmc],
        model_filtered_states=hmc_filtered_train.filtered_means,
        time_grid_forecast=t_emissions_train[start_idx_forecast_hmc:stop_idx_forecast_hmc],
        model_forecast_states=hmc_forecasted_train.forecasted_state_means,
        t_start=t_start,
        t_end=t_end,
        N_samples=hmc_N_samples,
    )

In [ ]:
# include initial params for comparison
plot_all_cdnlgssm_param_posteriors(
    param_samples=hmc_param_samples,
    param_properties=learnable_props,
    init_params=initial_learnable_params,
    true_params=true_params,
    )



In [ ]:
# omit initial params for easy viewing of region of high probability
plot_all_cdnlgssm_param_posteriors(
    param_samples=hmc_param_samples,
    param_properties=learnable_props,
    # init_params=initial_learnable_params,
    true_params=true_params,
)